In [57]:
import requests
import datetime
import time

server_url = "http://localhost:5000/sensor_data"

# Funzione per generare timestamp con intervalli di 30 secondi
def generate_timestamps(start_time, end_time):
    current_time = start_time
    timestamps = []
    while current_time <= end_time:
        timestamps.append(current_time.isoformat() + "Z")  # Formato ISO 8601 UTC
        current_time += datetime.timedelta(seconds=30)
    return timestamps

# Funzione per inviare i dati sintetici con "value" e "time_custom" come campi separati
def send_synthetic_data():
    data_points = []

    # 1° intervallo: 18:00 - 21:00 (nel letto)
    start = datetime.datetime(2024, 1, 29, 18, 0, 0)
    end = datetime.datetime(2024, 1, 29, 21, 0, 0)
    for t in generate_timestamps(start, end):
        data_points.append({"pressure_value": 4095, "time_custom": t}) #time_custom

    # 2° intervallo: 21:00 - 22:00 (fuori dal letto)
    start = datetime.datetime(2024, 1, 29, 21, 0, 30)
    end = datetime.datetime(2024, 1, 29, 22, 0, 0)
    for t in generate_timestamps(start, end):
        data_points.append({"pressure_value": 2000, "time_custom": t}) #time_custom

    # 3° intervallo: 22:00 - 23:30 (nel letto)
    start = datetime.datetime(2024, 1, 29, 22, 0, 30)
    end = datetime.datetime(2024, 1, 29, 23, 30, 0)
    for t in generate_timestamps(start, end):
        data_points.append({"pressure_value": 4095, "time_custom": t}) #time_custom

    # Invio dei dati al server Flask
    for payload in data_points:
        response = requests.post(server_url, json=payload)
        print(f"Sent {payload} at {payload['time_custom']}, Response: {response.status_code}") #time_custom
        time.sleep(0.01)

# Invia i dati sintetici
send_synthetic_data()

Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:00:00Z'} at 2024-01-29T18:00:00Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:00:30Z'} at 2024-01-29T18:00:30Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:01:00Z'} at 2024-01-29T18:01:00Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:01:30Z'} at 2024-01-29T18:01:30Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:02:00Z'} at 2024-01-29T18:02:00Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:02:30Z'} at 2024-01-29T18:02:30Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:03:00Z'} at 2024-01-29T18:03:00Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:03:30Z'} at 2024-01-29T18:03:30Z, Response: 200
Sent {'pressure_value': 4095, 'time_custom': '2024-01-29T18:04:00Z'} at 2024-01-29T18:04:00Z, Response: 200
Sent {'pressure_value': 4095

In [67]:
from influxdb_client import InfluxDBClient
import datetime
import os

# Configurazione di InfluxDB
INFLUXDB_URL = "http://localhost:8086"
TOKEN = os.getenv("token") # Sostituisci con il tuo token reale
ORG = "IotAlarmSystem"
BUCKET = "Prova"

client = InfluxDBClient(url=INFLUXDB_URL, token=TOKEN, org=ORG)
delete_api = client.delete_api()

# Intervallo di tempo per eliminare tutti i dati
start_time = "1970-01-01T00:00:00Z"  # Inizio dei tempi
end_time = datetime.datetime.utcnow().isoformat() + "Z"  # Ora attuale

# Predicate corretto per eliminare TUTTI i dati di una measurement
predicate = '_measurement="pressure"'  # Elimina tutti i dati della measurement "pressure"

try:
    delete_api.delete(start=start_time, stop=end_time, bucket=BUCKET, org=ORG, predicate=predicate)
    print(f"✅ Tutti i dati della measurement 'pressure' nel bucket '{BUCKET}' sono stati eliminati con successo!")
except Exception as e:
    print(f"❌ Errore durante l'eliminazione dei dati: {e}")


✅ Tutti i dati della measurement 'pressure' nel bucket 'Prova' sono stati eliminati con successo!
